In [125]:
from model import Encoder, Decoder, Seq2Seq
from data_loader import *
import pandas as pd
import torch.optim.lr_scheduler as lr_scheduler
from torch import optim
import torch.nn.functional as F
import datetime
import pretty_midi
import glob

In [126]:
import os
import matplotlib
import math
matplotlib.use('Agg')
# matplotlib.use("QtAgg")
import ffmpeg
#conda install -c conda-forge ffmpeg-python

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, writers
plt.rcParams['animation.ffmpeg_path'] = '/home/ilc/anaconda3/bin/ffmpeg'#'/usr/bin/ffmpeg'

import numpy as np
import subprocess as sp
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip

from midi2audio import FluidSynth

from torch.autograd import Variable 

In [127]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [128]:
dataset_name_path = f"./midi_list.txt"
dataloader = get_dataloader(dataset_name_path, batch_size=44) #[20, 512, 128], [20, 512, 102]

learning_rate = 0.001#0.001

input_size_encoder = 128 #129 #128
input_size_decoder = 102 #102 #24
output_size = 102#102 #24

# encoder_embedding_size = 300
# decoder_embedding_size = 300
enc_dropout = 0.5
dec_dropout = 0.
step = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

dataset_len 6600
cuda:0


In [129]:
class LSTM1(nn.Module):
    def __init__(self, output_dim, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.output_dim = output_dim #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, output_dim) #fully connected to determine output dim

        self.relu = nn.ReLU()

    def forward(self,x):
        # h0, c0 no time information
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state
        # Propagate input through LSTM
        # x is MIDI => [44, 512, 128]

        # hn is final state, run over the sequence length
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        # hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        # print("output.shape", output.shape)
        # print("hn.shape", hn.shape)
        # out = self.relu(hn)
        out = self.fc_1(output) #final
        return out
 

In [130]:
# Define the model architecture
input_size = 128 #number of features
hidden_size = 256 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
seq_len = 512
output_dim = 102 #number of output classes

# model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
# model = LSTM(embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
model = LSTM1(output_dim, input_size, hidden_size, num_layers, seq_len).to(device) #our lstm class
model.train()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

num_epochs = 100 #10
avg_loss_list = []
all_loss_list = []

In [131]:
def customized_mse_loss(output, target, prev_output, midi_array):
    # target = target.transpose(0, 1)
    # print("output", output)
    # print(output.shape) #torch.Size([20, 513, 102])
    # print(target.shape) #torch.Size([20, 513, 102])
    mse_loss = F.mse_loss(output, target)
    # print("mse_loss:", mse_loss)

    var_diff = torch.var(torch.squeeze(output), dim=1, keepdim=True)
    mean_diff = torch.mean(var_diff)
    
    # Condition 1: Penalize if output is similar to previous output
    if mean_diff < 1e-4: #threshold
        #output [512, 1, 102] => [102] <-> [102] <-> [102] <-> ... <-> [102]
        mse_loss *= 1000

    # Condition 2: Stop movement if input is all zeros
    # midi_transpose = midi_array.transpose(0, 1)
    # midi_sum_row = torch.sum(midi_transpose, dim=-1)
    # mask = midi_sum_row == 0
    # mask = mask.unsqueeze(-1)
    # mask = mask.to(device)
    # # according to recorded index, make a mask [0, 1, 1, 0, ..., 0], true part will be omit(set value to 0).
    # # before compute mse, use mask first to tensor, then caculate MES loss
    # masked_output = output.masked_fill(mask, 0) #inplace function
    # masked_target = target.masked_fill(mask, 0)
    # mse_loss += F.mse_loss(masked_output, masked_target) * 100 #output 和 previous output 不像的話，增大 loss

    # Condition 3: Penalize if right-hand movement is too different between outputs
    # if output.shape[-1] == 21:  # Assumes hand joints are the last 21 dimensions
    #     rh_indices = [i for i in range(12, 21)]  # Right-hand joint indices
    #     rh_output = output[..., rh_indices]
    #     rh_prev_output = prev_output[..., rh_indices]
    #     rh_loss = nn.functional.mse_loss(rh_output, rh_prev_output)
    #     if rh_loss > 0.1:
    #         mse_loss *= 1000

    return mse_loss

In [132]:
# train the model
for epoch in range(num_epochs):
    losses = []
    previous_output = torch.zeros(1, 512, 102).to(device)

    for i, (midi_batch, motion_batch) in enumerate(dataloader):
        model.train()
        
        midi_batch = midi_batch.to(device).float()
        motion_batch = motion_batch.to(device).float()
        # print("midi_batch", midi_batch.shape)
        # print("motion_batch", motion_batch.shape)

        optimizer.zero_grad()
        output = model(midi_batch) #midi_batch
        # print("output.shape", output.shape)

        # motion_ground_truth_padding = F.pad(motion_batch, (0,0,0,1), value = 1) #<eot>
        
        # loss =  F.mse_loss(output, motion_ground_truth_padding)
        loss =  F.mse_loss(output, motion_batch)
        # loss = customized_mse_loss(output, motion_ground_truth_padding, previous_output, midi_batch)
        # loss = customized_mse_loss(output, motion_batch, previous_output, midi_batch)

        # losses 累計lose
        losses.append(loss.cpu().item())
        all_loss_list.append(loss.cpu().item())
        loss.backward()

        optimizer.step()
        mean_loss = sum(losses)/len(losses)

        print(f"Epoch {epoch}, batch {i}: loss = {loss.cpu().item():.4f}")

        # scheduler.step(1)
    
        previous_output = output
        
        loc_dt = datetime.datetime.today()
        loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")
        # save_best_model(
        #     val_loss, epoch, model, optimizer, loss, loc_dt_format, mean_loss
        # )

    # val_loss = validate(model, val_dataloader) #CUDA out of memory
    # val_loss_list.append(val_loss)
    # save_best_model(
    #         val_loss, epoch, model, optimizer, loss, loc_dt_format, mean_loss
    #     )
    avg_loss_list.append(mean_loss)
    loc_dt = datetime.datetime.today()
    loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")
    if (epoch+1)%1000 == 0:
        torch.save({
            'epoch':epoch,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict(),
            'loss':loss
        }, "./model_save/[100epoch]LSTM_1LSTMenc_1LSTMdec_save_epoch_" + str(epoch)+ "_"+ str(loc_dt_format) + "_avg_loss_" + str(mean_loss) +".tar")

Epoch 0, batch 0: loss = 0.1698
Epoch 0, batch 1: loss = 0.1532
Epoch 0, batch 2: loss = 0.1269
Epoch 0, batch 3: loss = 0.9176
Epoch 0, batch 4: loss = 0.1078
Epoch 0, batch 5: loss = 0.1291
Epoch 0, batch 6: loss = 0.1376
Epoch 0, batch 7: loss = 0.1403
Epoch 0, batch 8: loss = 0.1407
Epoch 0, batch 9: loss = 0.1412
Epoch 0, batch 10: loss = 0.1396
Epoch 0, batch 11: loss = 0.1352
Epoch 0, batch 12: loss = 0.1303
Epoch 0, batch 13: loss = 0.1213
Epoch 0, batch 14: loss = 0.1024
Epoch 0, batch 15: loss = 0.0655
Epoch 0, batch 16: loss = 0.0199
Epoch 0, batch 17: loss = 0.0148
Epoch 0, batch 18: loss = 0.0259
Epoch 0, batch 19: loss = 0.0170
Epoch 0, batch 20: loss = 0.0074
Epoch 0, batch 21: loss = 0.0050
Epoch 0, batch 22: loss = 0.0065
Epoch 0, batch 23: loss = 0.0084
Epoch 0, batch 24: loss = 0.0095
Epoch 0, batch 25: loss = 0.0094
Epoch 0, batch 26: loss = 0.0084
Epoch 0, batch 27: loss = 0.0069
Epoch 0, batch 28: loss = 0.0056
Epoch 0, batch 29: loss = 0.0041
Epoch 0, batch 30: l

In [133]:
print(loc_dt_format)
print(avg_loss_list)

2023-07-02_04-57-15
[0.021721686967648566, 0.0017377050352903704, 0.0015704837899344663, 0.0014791829860769212, 0.001412314186648776, 0.0013605966915686924, 0.0013048289910269281, 0.0012735248209598165, 0.0012281012302264571, 0.0012038334901444613, 0.0011578531935811042, 0.001137808186467737, 0.0011079166484220574, 0.001091199442744255, 0.0010678352043032647, 0.001063957839893798, 0.0010439265763852745, 0.0010333356684229026, 0.0010085540059177826, 0.0009954549255780875, 0.0009742915645862619, 0.0009428519957388441, 0.000921420887655889, 0.0008943249719838302, 0.0008665721607394516, 0.0008304908165397743, 0.0007885878758194546, 0.0007555405523938437, 0.0007376392313744873, 0.0006910666807865103, 0.0006783294297444324, 0.0006283330529307326, 0.0006024248308191697, 0.0005658133182441815, 0.0005452072143089026, 0.0005088374744324634, 0.0004789982377163445, 0.00046748438733629883, 0.0004349014613156517, 0.0004287976399064064, 0.0004116912527630726, 0.0003901133583470558, 0.0003673975348162

In [134]:
# def lr_lambda(epoch):
#     # LR to be 0.1 * (1/1+0.01*epoch)
#     base_lr = 0.1
#     factor = 0.01
#     return base_lr/(1+factor*epoch)

In [135]:
# scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.3, total_iters=10)
# scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda)

In [136]:
plt.cla()
plt.clf()

In [137]:
print(len(avg_loss_list))
avg_loss_list_dataframe = pd.DataFrame(avg_loss_list)

100


In [138]:
avg_loss_list_dataframe

,0
0,0.021722
1,0.001738
2,0.001570
3,0.001479
4,0.001412
...,...
95,0.000139
96,0.000151
97,0.000165
98,0.000141


In [139]:
plt.plot(np.array(avg_loss_list_dataframe.index), np.array(avg_loss_list_dataframe[0]))
plt.savefig("avg_loss_training.jpg")
plt.show()

In [140]:
loss_list_dataframe = pd.DataFrame(all_loss_list)

In [141]:
plt.plot(np.array(loss_list_dataframe.index), np.array(loss_list_dataframe[0]))
plt.savefig("training_loss.jpg")
plt.show()

In [142]:
def predict(model, input, target, device):
    model.eval()
    with torch.no_grad():
        input = torch.as_tensor(input).to(torch.float32).to(device)
        print(target.shape)
        target = torch.as_tensor(target).to(torch.float32).to(device)
        # TODO: target should be <sos>, should not random
        outputs = model(input)
        return outputs.cpu().numpy()

In [143]:
def read_midi(filename, specific_fps):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(filename)

    piano_roll = midi_data.get_piano_roll(fs=specific_fps)  # 40fps #250fps
    piano_roll[piano_roll > 0] = 1

    return piano_roll

In [144]:
test_datapath = "./BWV1001/"
change_fps = 40
test_midi_path_list = glob.glob(test_datapath + "*.mid")
test_data_list = []
test_music_list = []
for test_midi in test_midi_path_list:
    str_name = test_midi
    print("str_name:", str_name)
    filename = str_name.split('/')[2]
    filecode = filename.split('.')[0]
    print("filecode: ",filecode)
    test_music_list.append(filecode)
    
    print(test_midi)
    read_piano_roll = read_midi(test_midi, change_fps)
    read_piano_roll_transpose = read_piano_roll.T
    print(read_piano_roll_transpose.shape)
    test_midi_len = read_piano_roll_transpose.shape[0]
    test_data_list.append(read_piano_roll_transpose)

str_name: ./BWV1001/vs1-1ada.mid
filecode:  vs1-1ada
./BWV1001/vs1-1ada.mid
(8171, 128)
str_name: ./BWV1001/vs1-2fug.mid
filecode:  vs1-2fug
./BWV1001/vs1-2fug.mid
(11537, 128)
str_name: ./BWV1001/vs1-3sic.mid
filecode:  vs1-3sic
./BWV1001/vs1-3sic.mid
(6993, 128)
str_name: ./BWV1001/vs1-4prs.mid
filecode:  vs1-4prs
./BWV1001/vs1-4prs.mid
(7897, 128)


In [145]:
def column(matrix, i):
    return [row[i] for row in matrix]

def test_render_animation(fps, output, azim, prediction, ground_truth=None):
    prediction_array = np.asarray(prediction)
    print(prediction_array.size)
    limit = len(prediction_array)
    print("limit", limit)
    size = 6#6
    fps = 40

    # Skeleton layout
    parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                [8, 18], [8, 20], [13, 21], [13, 19],
                [5, 20], [5, 21], [7, 18], [7, 19],
                [18, 19], [19, 21], [21, 20], [20, 18], #waist
                [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
    # joints_right = [1, 2, 12, 13, 14]

    prediction_array[:, :, 2] += 0.1 #[:, :, 2]
    if ground_truth is not None:
        ground_truth[:, :, 2] += 0.1
        poses = {'Prediction': prediction_array,
                 'Ground_truth': ground_truth}
    else:
        poses = {'Prediction': prediction_array}
    

    fig = plt.figure()#(figsize=(size*len(poses), size))
    # ax_3d = []
    # lines_3d = []
    radius = 1#14 #3.7#
    # print(poses)
    for index, (title, data) in enumerate(poses.items()):
        ax = fig.add_subplot(1, len(poses), index + 1, projection='3d')
        ax.clear()
        print(data)
        ims = [] #每一 frame 都存
        for frame_index, each_frame in enumerate(data):
            # print("each_frame")
            # print(each_frame)
            ax.view_init(elev=15., azim=azim)
            ax.set_xlim3d([-radius/2, radius/2])
            ax.set_zlim3d([0, radius])
            ax.set_ylim3d([-radius/2, radius/2])
            ax.set_aspect('auto') #ax.set_aspect('equal')

            # print(title)
            points = ax.scatter(column(each_frame[:30], 0), column(each_frame[:30], 1), column(each_frame[:30], 2), cmap='jet', marker='o', label='body joint', color = 'black')
            points_2 = ax.scatter(column(each_frame[30:32], 0), column(each_frame[30:32], 1), column(each_frame[30:32], 2), cmap='jet', marker='o', label='body joint', color = 'blue')
            points_3 = ax.scatter(column(each_frame[32:34], 0), column(each_frame[32:34], 1), column(each_frame[32:34], 2), cmap='jet', marker='o', label='body joint', color = 'red')
            
            # ax.scatter(column(each_frame, 0), column(each_frame, 1), column(each_frame, 2), cmap='jet', marker='o', label='body joint')
            # ax.legend()
            # print("+++")
            
            parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                        [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                        [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                        [8, 18], [8, 20], [13, 21], [13, 19],
                        [5, 20], [5, 21], [7, 18], [7, 19],
                        [18, 19], [19, 21], [21, 20], [20, 18], #waist
                        [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                        [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                        [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                        [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                        [30, 31], [32, 33],  #instrument
                        # [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
            lines = []
            # draw line
            
            # lines = [ax.plot([each_frame[vs][0], each_frame[ve][0]],
            #                  [each_frame[vs][1], each_frame[ve][1]],
            #                  [each_frame[vs][2], each_frame[ve][2]]) for (vs, ve) in parents]
            line_num = len(parents)
            for idx, each_line in enumerate(parents):
                vec_start = each_frame[each_line[0]]
                vec_end = each_frame[each_line[1]]
                # print(vec_start)
                # print(vec_end)
                line_color = "black"
                if idx == line_num-2:
                    line_color = "blue"
                if idx == line_num-1:
                    line_color = "red"
                # ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]])
                
                temp, = ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]], color=line_color)
                lines.append(temp)

            # ax.figure.savefig('./test_pic/pic' + str(frame_index) + '.png', dpi=100, bbox_inches = 'tight')

            # ims.append([points])
            # image_frame = [points].extend(lines)
            ims.append([points]+[points_2]+[points_3]+lines) #TODO: try extend

            # plt.cla()
            # print("+++")

    anim = matplotlib.animation.ArtistAnimation(fig, ims, interval=1000/fps)

    if output.endswith('.mp4'):
        FFwriter = matplotlib.animation.FFMpegWriter(fps=fps, extra_args=['-vcodec', 'libx264'])
        anim.save(output, writer=FFwriter)
    elif output.endswith('.gif'):
        anim.save(output, fps=fps, dpi=100, writer='imagemagick')
    else:
        raise ValueError('Unsupported output format (only .mp4 and .gif are supported)')

In [146]:
def plot(audio_path, plot_path, prediction, sample_time, fps, name=""): #audio_path, plot_path, 
    # render_animation(fps, output='new_temp.mp4', azim=75, prediction=prediction)
    test_render_animation(fps, output='new_temp_' + name + '.mp4', azim=75, prediction=prediction)

    # # #merge with wav
    input_video = ffmpeg.input('new_temp_' + name + '.mp4')
    fluid_syn = FluidSynth()
    fluid_syn.midi_to_audio(audio_path, './output' + name + '.wav')
    input_audio = ffmpeg.input('./output' + name + '.wav')
    # output = ffmpeg.output(video, audio, plot_path, vcodec='copy', acodec='aac', strict='experimental')
    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(plot_path).run()
    # os.remove('new_temp_' + name + '.mp4')

In [147]:
model.eval()

full_prediction = pd.DataFrame()
num_count = 0
# read midi
# test_dataloader = get_dataloader(test_datapath, batch_size=1)
for test_batch in test_data_list:
    with torch.no_grad():
        first_target = torch.zeros(test_batch.shape[0],102)
        print(first_target.shape)
        test_input = test_batch[None, :]
        test_target = first_target[None, :]
        print("test_input", test_input.shape)
        print("test_target", test_target.shape)
        prediction = predict(model, test_input, test_target, device)
        
        # print(prediction.shape)
        
        prediction  = prediction[:, :, :102]
        print("prediction.shape", prediction.shape)
        
        # full_prediction.append(prediction)
        full_prediction = pd.DataFrame(prediction[0])
        print("full_prediction", full_prediction.shape)
        
        # prev_prediction = prediction[0][:-1][None, :]
        # print(prev_prediction.shape)
        
        Row_list_prediction =[]
        
        filecode = test_music_list[num_count]
    
        # Iterate over each row
        for index, rows in full_prediction.iterrows():
            #fill nan
            rows = rows.fillna(0)
            # Create list for the current row
            my_list = rows.values.tolist()
            # print(my_list)
            
            my_list_per3 = [my_list[i:i+3] for i in range(0, len(my_list), 3)]
            # append the list to the final list
            Row_list_prediction.append(my_list_per3)
        
        # print(len(Row_list_prediction), len(Row_list_prediction[0]),len(Row_list_prediction[0][0]))
        plot(test_datapath + test_music_list[num_count] + ".mid", "./video_" + filecode + "_test_predict.mp4", Row_list_prediction[:800], None, 40, filecode) #ow_list[0:900]
        # print("prediction.shape", prediction.shape)

        # formated_motion = prediction_format(full_prediction)
        # # # plot(formated_motion)
        # audio_path = test_music_list[num_count][0]
        # output_path = "test_output_" + filecode + ".mp4"
        # plot(formated_motion, audio_path, output_path, None, 10, filecode)
        num_count += 1

# model.train()

torch.Size([8171, 102])
test_input (1, 8171, 128)
test_target torch.Size([1, 8171, 102])
torch.Size([1, 8171, 102])
prediction.shape (1, 8171, 102)
full_prediction (8171, 102)


81600
limit 800
[[[ 0.04195466  0.11844039  1.08897183]
  [-0.01673407  0.07854448  1.09859905]
  [ 0.0914432   0.05186868  1.09599016]
  ...
  [ 0.00706922  0.07912512  0.96359668]
  [-0.17392084  0.04858348  1.05985425]
  [ 0.10691226  0.22179675  0.78177718]]

 [[ 0.0328354   0.1244849   1.08951489]
  [-0.02532874  0.08513726  1.10001025]
  [ 0.08211606  0.06168494  1.10144684]
  ...
  [-0.00164157  0.08511345  0.97257272]
  [-0.1661277   0.07697129  1.06210331]
  [ 0.10458596  0.21888529  0.75777016]]

 [[ 0.0269833   0.12925099  1.09494696]
  [-0.03233646  0.08849356  1.10562727]
  [ 0.07739699  0.06651065  1.10809002]
  ...
  [-0.00799266  0.08912583  0.97735689]
  [-0.17746456  0.08872232  1.08135823]
  [ 0.09012791  0.21607313  0.76037065]]

 ...

 [[ 0.04122292  0.1139506   1.09419463]
  [-0.01042086  0.05767747  1.10526762]
  [ 0.10716971  0.06427822  1.10226748]
  ...
  [ 0.01119085  0.07083274  0.97384021]
  [-0.15326339  0.0533957   1.07971523]
  [ 0.0767303   0.21574782  

fluidsynth: panic: An error occurred while reading from stdin.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file './outputvs1-1ada.wav'..


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

...

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

torch.Size([7897, 102])
test_input (1, 7897, 128)
test_target torch.Size([1, 7897, 102])
torch.Size([1, 7897, 102])
prediction.shape (1, 7897, 102)
full_prediction (7897, 102)
81600
limit 800
[[[ 0.050954    0.10515174  1.10352347]
  [-0.01275526  0.06568285  1.10952029]
  [ 0.09779105  0.03756712  1.10947738]
  ...
  [ 0.01192863  0.08291004  0.97720537]
  [-0.10606502  0.11452394  1.06592212]
  [ 0.10787617  0.1897625   0.73426054]]

 [[ 0.05220299  0.10302494  1.09884349]
  [-0.01065362  0.06449585  1.10469625]
  [ 0.09790954  0.03542342  1.10275111]
  ...
  [ 0.01230039  0.08450968  0.97191415]
  [-0.10556395  0.11553828  1.06773249]
  [ 0.10589424  0.18719152  0.73299787]]

 [[ 0.0515597   0.10284261  1.09928427]
  [-0.01177683  0.0644156   1.10546694]
  [ 0.09805752  0.03584164  1.10220037]
  ...
  [ 0.01143601  0.08507293  0.97114668]
  [-0.11164292  0.11249834  1.07258222]
  [ 0.10269383  0.18806869  0.73646817]]

 ...

 [[ 0.01224583  0.11963961  1.09995327]
  [-0.05063241  0.

fluidsynth: panic: An error occurred while reading from stdin.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file './outputvs1-4prs.wav'..


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55